In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from app_store_scraper import AppStore
from pprint import pprint


In [6]:
import requests
import random

query = "apple pie"

r = requests.get("https://api.qwant.com/v3/search/images",
    params={
        'count': 50,
        'q': query,
        't': 'images',
        'safesearch': 1,
        'locale': 'en_US',
        'offset': 0,
        'device': 'desktop'
    },
    headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    }
)

response = r.json().get('data').get('result').get('items')


In [7]:
response

{'mainline': [{'type': 'web',
   'count': 10,
   'items': [{'title': 'Morup apple pie Matzusah magumu lecsek pig je.',
     'favicon': '',
     'url': 'http://wec.ma/wuzudo',
     'source': 'http://de.pl/gabmuij',
     'desc': 'Bi ogeuz tusgi ukugisi jal sowugici netaafe nahozgog fojvuope fogo puozov jepjojsul ku otriov po ta juha boc. Iku pu wugicov pol ipkufeg aragib pinjef ge lanuwuv hi jeoke ewjof evawah ku ehi pioz. Purfoure ubu webkimzem feb otooh jedu opto va tanop ce huwra sasoseze.',
     '_id': '86c85ae05ea4ddb8c42962761f0b2bf5'},
    {'title': 'Go hiheebo luteb apple pie Siuc fiv luw.',
     'favicon': '',
     'url': 'http://ajuvoh.af/iwufoti',
     'source': 'http://jakgovcet.fk/eh',
     'desc': 'Vow ji pi mabsud koiwcad we loftazaka fupovum elubawu sirfobu vabuso mi jojeli lendaj co nirsah. Nofohipo ka pesa celevzi diapsug caskomi irufoje dodzep va pasaf fozkic azebozlaw epihu ez gutib re ci. Vohrenvu igziwig bitonkuj edo lismu inva lisjic jatmucvo fadso na uz cefungam h

In [ ]:

urls = [r.get('media') for r in response]
print(random.choice(urls))

In [6]:
from app_store_scraper import *

In [24]:
a = AppStore('USA','MyFitnessPal: Calorie Counter')

2022-04-11 18:43:59,022 [INFO] Base - Searching for app id
2022-04-11 18:44:00,579 [INFO] Base - Initialised: AppStore('usa', 'myfitnesspal-calorie-counter', 341232718)
2022-04-11 18:44:00,580 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/usa/app/myfitnesspal-calorie-counter/id341232718


In [25]:
a.review()

2022-04-11 18:44:03,205 [ERROR] Base - Something went wrong: 'data'
2022-04-11 18:44:03,205 [INFO] Base - [id:341232718] Fetched 0 reviews (0 fetched in total)


In [22]:
a.reviews_count

0

In [ ]:

minecraft = AppStore(country="nz", app_name="minecraft")
minecraft.review(how_many=20)

pprint(minecraft.reviews)
pprint(minecraft.reviews_count)

In [2]:
!pip install play-store

In [5]:
!pip install lxml

  Using cached lxml-4.8.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)


In [8]:
from  play_store import *

app = App('com.android.chrome')

InvalidSchema: No connection adapters were found for '/store/apps/collection/cluster?clp=ogogCBEqAggIMhgKEmNvbS5hbmRyb2lkLmNocm9tZRABGAM%3D:S:ANO1ljI-7S4&gsr=CiOiCiAIESoCCAgyGAoSY29tLmFuZHJvaWQuY2hyb21lEAEYAw%3D%3D:S:ANO1ljL4keg'

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_recreate_milvus.ipynb.
Converted 00_tools.ipynb.
Converted 02_onehot.ipynb.
Converted 03_quantization.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted milvus_dev.ipynb.
